# Pracitice Exercise

In [ ]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
import os

print("All libraries imported!")

In [ ]:
sys_msg = SystemMessage(
    content = "Your are a polite and helpful customer support rep. Answer customers' questions accurately and be concise."

)

def sales_assist(state: MessagesState) -> dict:
    """
    The assistant node - processes messages and generates response.
    """
    messages = [sys_msg] + state["messages"]

    response = llm.invoke(messages)
    return {"messages": [AIMessage(content=response.content)]}

print("Sales assistant mode defined")

# The graph
builder = StateGraph(MessagesState)
builder.add_node("assistant", sales_assist)

builder.add_edge(START, "assistant")
builder.add_edge("assistant", END)

#Checkpointer
memory = MemorySaver()

agent2 = builder.compile(checkpointer=memory)
print('Agent compiled with memory')

In [ ]:
def interactive_chat():
    """
    Run an interactive chat session.
    Type 'exit' or 'quit' to stop.
    """
    print("\n" + "="*70)
    print("Interactive Chat Started")
    print("Type your message and press Enter. Type 'exit' to quit.")
    print("="*70 + "\n")

    thread_id = "interactive_session2"

    while True:
        user_input = input("\nYou: ").strip()
        if user_input.lower() in ["exit", "quit"]:
            print("\nGoodbye...")
            break
        
        if not user_input:
            continue
        
        #Get response
        result = agent2.invoke(
            {"messages": [HumanMessage(content=user_input)]},
            config={"configurable": {"thread_id": thread_id}}
        )

        # Print response
        agent_message = result["messages"][-1]
        user_message = result['messages'][-2]
        print(f"\nYou: {user_message.content}")
        print(f"\nAgent: {agent_message.content}")

interactive_chat()